In [1]:
import sys
sys.path.append("/Users/jisu/Desktop/dev/cli/av2av")
from unit2unit.inference import load_model as load_unit2unit_model

utut_path = "/Users/jisu/Desktop/dev/cli/av2av/modelckpt/utut_sts_ft.pt"
src_lang = "en"
tgt_lang = "es"
use_cuda = False

unit2unit_task, unit2unit_generator = load_unit2unit_model(utut_path, src_lang, tgt_lang, use_cuda=use_cuda)

2026-01-18 20:38:44 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
/Users/jisu/Desktop/dev/cli/av2av/fairseq/fairseq/checkpoint_utils.py:322: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on 

loaded model config: {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': 'tblog', 'wandb_project': None, 'azureml_logging': False, 'seed': 2, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': './', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post

In [5]:
import argparse
import numpy as np
import torch

from fairseq import checkpoint_utils, utils
from fairseq_cli.generate import get_symbols_to_strip_from_output

from unit2unit.task import UTUTPretrainingTask
from util import process_units, save_unit

models, cfg, task = checkpoint_utils.load_model_ensemble_and_task([utut_path])
print(f"loaded model config: {cfg}")

model = models[0]
trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)

total_params = sum(
    p.numel() for p in model.parameters()
)

print(f"Trainable params: {trainable_params:,}")
print(f"Total params:     {total_params:,}")


/Users/jisu/Desktop/dev/cli/av2av/fairseq/fairseq/checkpoint_utils.py:322: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_location=torch.device("cpu

loaded model config: {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': 'json', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': 'tblog', 'wandb_project': None, 'azureml_logging': False, 'seed': 2, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': './', 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post

In [ ]:
unit2unit_task

In [5]:
unit2unit_generator

SequenceGenerator(
  (model): EnsembleModel(
    (single_model): BARTModel(
      (encoder): TransformerEncoderBase(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(1024, 1024, padding_idx=1)
        (embed_positions): SinusoidalPositionalEmbedding()
        (layers): ModuleList(
          (0-11): 12 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (fc1

In [24]:
import torch

utut_ckpt = torch.load(utut_path, map_location='cpu')


print(f"Number of weights: {len(list(utut_ckpt['model'].keys()))}")

# 가중치 키 목록 중 상위 20개만 출력하여 언어 코드가 포함되어 있는지 확인
for key in list(utut_ckpt['model'].keys())[:20]:
    print(key)

/var/folders/y9/nm4qdmtn143d63hcl5r54xbr0000gn/T/ipykernel_6665/2460976691.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  utut_ckpt = torch.load(utut_path, map_location

Number of weights: 515
encoder.version
encoder.embed_tokens.weight
encoder.embed_positions._float_tensor
encoder.layers.0.self_attn.k_proj.weight
encoder.layers.0.self_attn.k_proj.bias
encoder.layers.0.self_attn.v_proj.weight
encoder.layers.0.self_attn.v_proj.bias
encoder.layers.0.self_attn.q_proj.weight
encoder.layers.0.self_attn.q_proj.bias
encoder.layers.0.self_attn.out_proj.weight
encoder.layers.0.self_attn.out_proj.bias
encoder.layers.0.self_attn_layer_norm.weight
encoder.layers.0.self_attn_layer_norm.bias
encoder.layers.0.fc1.weight
encoder.layers.0.fc1.bias
encoder.layers.0.fc2.weight
encoder.layers.0.fc2.bias
encoder.layers.0.final_layer_norm.weight
encoder.layers.0.final_layer_norm.bias
encoder.layers.1.self_attn.k_proj.weight


In [25]:
utut_ckpt

{'args': None,
 'cfg': {'_name': None,
  'common': {'_name': None,
   'no_progress_bar': False,
   'log_interval': 100,
   'log_format': 'json',
   'log_file': None,
   'aim_repo': None,
   'aim_run_hash': None,
   'tensorboard_logdir': 'tblog',
   'wandb_project': None,
   'azureml_logging': False,
   'seed': 2,
   'cpu': False,
   'tpu': False,
   'bf16': False,
   'memory_efficient_bf16': False,
   'fp16': True,
   'memory_efficient_fp16': False,
   'fp16_no_flatten_grads': False,
   'fp16_init_scale': 128,
   'fp16_scale_window': None,
   'fp16_scale_tolerance': 0.0,
   'on_cpu_convert_precision': False,
   'min_loss_scale': 0.0001,
   'threshold_loss_scale': None,
   'amp': False,
   'amp_batch_retries': 2,
   'amp_init_scale': 128,
   'amp_scale_window': None,
   'user_dir': './',
   'empty_cache_freq': 0,
   'all_gather_list_size': 16384,
   'model_parallel_size': 1,
   'quantization_config_path': None,
   'profile': False,
   'reset_logging': False,
   'suppress_crashes': False

In [7]:
unit2av_path='/Users/jisu/Desktop/dev/cli/av2av/modelckpt/unit_av_renderer.pt'

unit2a_ckpt = torch.load(unit2av_path, map_location='cpu')

# 가중치 키 목록 중 상위 20개만 출력하여 언어 코드가 포함되어 있는지 확인
# for key in list(unit2a_ckpt['model'].keys())[:20]:
#     print(key) >>> KeyError: 'model'

unit2a_ckpt['audio'].keys()

/var/folders/y9/nm4qdmtn143d63hcl5r54xbr0000gn/T/ipykernel_14319/1375809416.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unit2a_ckpt = torch.load(unit2av_path, map_lo

dict_keys(['en', 'es', 'fr', 'it', 'pt'])

오디오 CodeHiFiGANModel_spk 상속 구조:
```
CodeHiFiGANModel_spk
  ↳ CodeHiFiGANModel
    ↳ nn.Module

```

In [11]:
# 보코더는 “fairseq 모델 1개”가 아니라, 여러 서브모델을 조합한 renderer/vocoder 파이프라인
from fairseq import utils
from unit2av.model import UnitAVRenderer
from unit2av.model_speaker_encoder import SpeakerEncoder
from util import save_video, extract_audio_from_video

with open('/Users/jisu/Desktop/dev/cli/av2av/unit2av/config.json') as f:
    vocoder_cfg = json.load(f)
vocoder = UnitAVRenderer(unit2av_path, vocoder_cfg, 'en')
 
model = vocoder
trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)

total_params = sum(
    p.numel() for p in model.parameters()
)

print("Unit2AV Vocoder param. size")
print(f"Trainable params: {trainable_params:,}")
print(f"Total params:     {total_params:,}")


/opt/anaconda3/envs/av2av/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/Users/jisu/Desktop/dev/cli/av2av/unit2av/model.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weig

Removing weight norm...
Unit2AV Vocoder param. size
Trainable params: 50,066,805
Total params:     50,066,805


In [12]:
vocoder

UnitAVRenderer(
  (model): CodeHiFiGANModel_spk(
    (conv_pre): Conv1d(256, 512, kernel_size=(7,), stride=(1,), padding=(3,))
    (ups): ModuleList(
      (0): ConvTranspose1d(512, 256, kernel_size=(11,), stride=(5,), padding=(3,))
      (1): ConvTranspose1d(256, 128, kernel_size=(8,), stride=(4,), padding=(2,))
      (2): ConvTranspose1d(128, 64, kernel_size=(8,), stride=(4,), padding=(2,))
      (3): ConvTranspose1d(64, 32, kernel_size=(4,), stride=(2,), padding=(1,))
      (4): ConvTranspose1d(32, 16, kernel_size=(4,), stride=(2,), padding=(1,))
    )
    (resblocks): ModuleList(
      (0): ResBlock(
        (convs1): ModuleList(
          (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
          (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
        )
        (convs2): ModuleList(
          (0-2): 3 x Conv1d(256, 256, kernel_size=(3,), s

---
### invest ckpts

In [2]:
import torch

def inspect_checkpoint(path):
    print(f"\n[Checking: {path}]")
    # 모델 불러오기 (가중치 제외하고 메타데이터 위주로 로드)
    ckpt = torch.load(path, map_location='cpu')
    
    # 1. 아키텍처 차원 확인 (BART/mBART 기준)
    cfg = ckpt.get('cfg', None)
    if cfg and hasattr(cfg, 'model'):
        model_cfg = cfg.model
        print(f"Encoder Dim: {model_cfg.encoder_embed_dim}")
        print(f"Decoder Dim: {model_cfg.decoder_embed_dim}")
        print(f"Encoder Layers: {model_cfg.encoder_layers}")
        print(f"Decoder Layers: {model_cfg.decoder_layers}")
        print(f"Attention Heads: {model_cfg.encoder_attention_heads}")
    
    # 2. 딕셔너리 크기 (임베딩 레이어의 행 개수로 파악)
    # state_dict 혹은 model 키에서 추출
    state_dict = ckpt.get('model', ckpt.get('state_dict', {}))
    embed_weight = state_dict.get('encoder.embed_tokens.weight', None)
    
    if embed_weight is not None:
        vocab_size, embed_dim = embed_weight.shape
        print(f"Vocabulary Size (Dictionary): {vocab_size}")
        print(f"Embedding Dim (Weight Matrix): {embed_dim}")
    else:
        print("Could not find embedding weights in checkpoint.")

# 두 모델 경로 입력
original_path = "/home/2022113135/jjs/av2av/unit2unit/utut_finetune/unit_mbart.pt"
finetuned_path = "/home/2022113135/jjs/av2av/unit2unit/utut_finetune/utut_ckpt/unit_mbart_multilingual_ft/en_ko/checkpoint_best.pt"

official_utut = "/home/2022113135/jjs/av2av/ckpts/utut_sts_ft.pt"

inspect_checkpoint(original_path)
inspect_checkpoint(finetuned_path)

inspect_checkpoint(official_utut)


[Checking: /home/2022113135/jjs/av2av/unit2unit/utut_finetune/unit_mbart.pt]
Vocabulary Size (Dictionary): 1007
Embedding Dim (Weight Matrix): 1024

[Checking: /home/2022113135/jjs/av2av/unit2unit/utut_finetune/utut_ckpt/unit_mbart_multilingual_ft/en_ko/checkpoint_best.pt]
Vocabulary Size (Dictionary): 1025
Embedding Dim (Weight Matrix): 1024

[Checking: /home/2022113135/jjs/av2av/ckpts/utut_sts_ft.pt]
Vocabulary Size (Dictionary): 1024
Embedding Dim (Weight Matrix): 1024
